#3 WELCOME TO LANGCHAIN

#3.0 LLMs and Chat Models

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI


llm = OpenAI()
chat =  ChatOpenAI()

a = llm.predict("How many planets are there?")

b = chat.predict("How many planets are there?")

a,b

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI


llm = OpenAI(OPENAI_API_KEY="sk-2ttttttttttttttttttttttttttttttttttttttttttttttt")
chat =  ChatOpenAI(OPENAI_API_KEY="sk-2ttttttttttttttttttttttttttttttttttttttttttttttt")

a= llm.predict("How many planets are there?")

b=chat.predict("How many planets are there?")

a,b

Best Models and Cheap

In [ ]:
from langchain.chat_models import ChatOpenAI


chat =  ChatOpenAI()

b=chat.predict("How many planets are there?")

b

#3.1 Predict Messages

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage


chat =  ChatOpenAI(temperature=0.1)

prompt =[
    SystemMessage(content="You are geography expert. And you only reply in Korean.",),
    AIMessage(content="내 이름은 OpenAI 깐부입니다."),
    HumanMessage(content="What is the distance between Korea and Japan. And What is your name?")
]

chat.predict_messages(prompt)

#3.2 Prompt Templates

Nomal Prompt Templates - PromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


chat =  ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template("What is the distance between {country_A} and {country_b}",)
prompt = template.format(country_A="Korea",country_b="Japan")

chat.predict(prompt) 

ChatPromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import  ChatPromptTemplate


chat =  ChatOpenAI(temperature=0.1)

template =ChatPromptTemplate.from_messages(
    [
    ("system", "You are geography expert. And you only reply in {language}."),
    ("ai","내 이름은 OpenAI {name}입니다."),
    (
        "human",
        "What is the distance between {country_A} and {country_b}. And What is your name?",
        )
    ]
)
prompt = template.format_messages(
    language="Korean",
    name = "Socrates",
    country_A="Korea",
    country_b="Japan"
    )

chat.predict_messages(prompt)

#3.3 OutputParser and LCEL(LangChain Expression Language)

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip,items))
    
p = CommaOutputParser()
p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser


chat =  ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip,items))

template =ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. DO NOT reply with anything else."),
    ("human","{question}")
])
prompt = template.format_messages(
    max_items=10,
    question = "What are the colors?"
    )

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

Make langchain getting shorter coding - Chain

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser


chat =  ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip,items))

template =ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. DO NOT reply with anything else."),
    ("human","{question}")
])

chain =  template | chat | CommaOutputParser()
chain.invoke({
    "max_items":10,
    "question":"What are the planets?"
})

chain1 =  template1 | chat1 | CommaOutputParser1() \n
chain2 =  template2 | chat2 | CommaOutputParser2() \n
chain =  chain1 | chain2 | OutputParses() \n
두개 이상의 chain을 연결 가능하다.

#3.4 Chaining Chains

https://python.langchain.com/docs/expression_language/interface

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat =  ChatOpenAI(
        temperature=0.1, 
        streaming=True, 
        callbacks=[
            StreamingStdOutCallbackHandler()
        ]
    )
chef_prompt =ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to folow recipies for any type of cuisine with easy to find ingredients."),
    ("human","I want to cook {cuisine} food.")
])
chef_chain =  chef_prompt | chat 

veg_chef_prompt =ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human","{recipe}")
])
veg_chef_chain =  veg_chef_prompt | chat

final_chain = {"recipe":chef_chain} | veg_chef_chain

final_chain.invoke({"cuisine":"Korean"})